In [1]:
import sys
sys.path.append('..')
from vtlc.notebook import *

In [2]:
(channels, _) = load_hololive()

In [4]:
annotation_threshold = 40600
avatar_size = 24
width = 800
height = 600
period = '2022-05'

def plot(data, line_color='red'):
    chart = Chart(data,
                  title=TitleParams(f'Live Chat Population ({period})',
                                    fontSize=25)).mark_image(
                                        cornerRadius=15,
                                        clip=True,
                                        width=avatar_size,
                                        height=avatar_size).encode(
                                            X('subscriptionCount',
                                              scale=alt.Scale(
                                                #   type='log',
                                                #   align=0.5,
                                                  bins=[
                                                      100000,
                                                      250000,
                                                      500000,
                                                      750000,
                                                      1000000,
                                                      1500000,
                                                      2000000,
                                                      4000000
                                                  ]),
                                              title='Subscribers',
                                              axis=Axis(format='~s')),
                                            Y(
                                                'uniqueChatters',
                                                # scale=alt.Scale(type='log'),
                                                title='Unique Chatters',
                                                axis=Axis(format='~s')),
                                            url='photo').properties(width=width,
                                                                    height=height)

    # trendline
    chart = chart + chart.transform_regression(
        'subscriptionCount', 'uniqueChatters',
        method='quad').mark_line(color=line_color)

    # annotation
    chart = chart + Chart(data).mark_text(
        align='left', baseline='middle', fontSize=20, dx=20, dy=-5).encode(
            X('subscriptionCount'),
            Y('uniqueChatters'),
            Text('uniqueChatters', format='~s'),
        ).transform_filter(
            datum['uniqueChatters'] > annotation_threshold) + Chart(
                data).mark_text(
                    align='left', baseline='middle', fontSize=10, dx=20,
                    dy=10).transform_calculate(
                        subc=f'{datum.subscriptionCount} + " Subs"').encode(
                            X('subscriptionCount'),
                            Y('uniqueChatters'),
                            Text('subscriptionCount', format='~s'),
                            # text=Text('subc', type='nominal')
                        ).transform_filter(
                            datum['uniqueChatters'] > annotation_threshold)

    return chart


def add_footer(chart):


    # img = alt.Chart({
    #     "values": [{"url": "https://holodata.org/logo.svg"}]
    # }).mark_image(opacity=1).encode(
    #     x=alt.value(0), x2=alt.value(30),  # pixels from left
    #     y=alt.value(height), y2=alt.value(height+30),  # pixels from top
    #     url="url:N"
    # )
    # chart = alt.layer(chart, img)
    def footer(chart, title, anchor='end'):
        return alt.concat(chart).properties(
            title=TitleParams(title,
                              anchor=anchor,
                              baseline='bottom',
                              orient='bottom',
                              fontWeight='normal',
                              fontSize=10))

    chart = footer(chart, [
        'Data Source: VTuber 1B',
        'https://holodata.org/vtuber-1b'
    ])

    chart = footer(chart, 'holodata.org', 'start')

    return chart


# subset = overall.sort_values('uniqueChatters', ascending=False).groupby('group').head(1).reset_index(drop=True)
subset = channels.query(f'period == "{period}"')

en = subset[subset['group'].str.contains('English')]
indonesia = subset[subset['group'].str.contains('Indonesia')]
jp = subset[~(subset['group'].str.contains('English')
              | subset['group'].str.contains('Indonesia'))]

chart = add_footer(
    plot(jp) + plot(indonesia, line_color='yellow') +
    plot(en, line_color='#8658FF'))

chart


alt.ConcatChart(...)